In [54]:
import os
import time


class FlowDataKit():
    
    def __init__(self):
        pass
    
    def get_flow_data(self, length, hosts=None, ports=None, protocal=None, return_back=None):

        if length != None:
            command = "tcpdump -c %s " % length
        else:
            command = "tcpdump "

        if hosts != None and type(hosts) == dict:
            supplement2 = []
            for host in list(hosts.keys()):
                if hosts[host] == "any":
                    supplement2.append("host %s " % host)
                else:
                    supplement2.append("%s %s " % (hosts[host], host))
            supplement3 = supplement2[0]
            for i in supplement2:
                if i == supplement2[0]:
                    continue
                supplement3 = supplement3 + "or " + i

        if ports != None and type(ports) == dict:
            supplement2 = []
            for port in list(ports.keys()):
                if ports[port] == "any":
                    supplement2.append("port %s " % port)
                else:
                    supplement2.append("%s port %s " % (ports[port], port))
            supplement4 = supplement2[0]
            for i in supplement2:
                if i == supplement2[0]:
                    continue
                supplement4 = supplement4 + "or " + i
            supplement3 = supplement3 + "and (" + supplement4 + ")"
        elif protocal != None:
            if protocal == "any":
                supplement4 = ""
            else:
                supplement4 = protocal
            supplement3 = "(%s) and %s" % (supplement3, supplement4)


        supplement = "-n '%s'" % supplement3
        command = command + supplement
        print("get_flow_info: ", command)

        if length != None:
            terminal = os.popen(command)
            result = terminal.read()
            terminal.close()

            result = result.split("\n").remove("")
    #         print("result", result)
            return result
        else:
            terminal = os.popen(command + " > /home/hadream/network_anti_tcpudmp.out 2>/home/hadream/network_anti_tcpudmp_error.out &")
            pid = terminal.read().replace("[1] ", "")
            print("pid", pid)
            terminal.close()
            
            time.sleep(5)
            
            file = open("/home/hadream/network_anti_tcpudmp.out", "r")
            feed = file.readline()
            while return_back(feed) != 1:
                time.sleep(0.5)
                feed = file.readline()
            os.system("kill -9 %s" % pid) # still a problem
            print("monitor end")
            file.close()

    def flow_data_analyze(self, data):

        outputs = []

        print("analyzing flow data...")
        count = 0
        for i in data:
            count+=1
#             if count == len(data):
#                 break
            output = {
                "id": "",
                "who": {"src": "", "src_port": "", "dst": "", "dst_port": ""},
                "what": {"timestamp": "", "flags": "", "ack": "", "seq": [0, 1], "length": ""},
                "more": {"options": []}
            }

            print("analyzing data-%s" % count)
            output["id"] = count
            # phase 1
            part = i.split(": ")
    #         print(part)
            part1 = part[0].split(" ", 2)
            part2 = part[1].split(", ")
    #         print(part1, "\n", part2)
            part3 = part1[2].split(" > ")
    #         print(part3)
            a, b = part3[0].rsplit(".", 1), part3[1].rsplit(".", 1)
            output["who"]["src"], output["who"]["src_port"] = a[0], a[1]
            output["who"]["dst"], output["who"]["dst_port"] = b[0], b[1]
    #         print(src_ip, src_port)
    #         print(dst_ip, dst_port)
            output["what"]["timestamp"] = part1[0]
            for j in part2:
                if "Flags" in j:
                    output["what"]["flags"] = j.split(" ")[1]
                elif "seq" in j:
                    output["what"]["seq"] = j.split(" ")[1].split(":")
                elif "ack" in j:
                    output["what"]["ack"] = j.split(" ")[1]
                elif "options" in j:
                    output["more"]["options"] = j.split(" ", 1)[1]
                elif "length" in j:
                    output["what"]["length"] = j.split(" ")[1]
            outputs.append(output)

        return outputs
        
flow_data_kit = FlowDataKit()
flow_data_kit.flow_data_analyze(flow_data_kit.get_flow_data(None, {"8.130.174.186": "any"}, None, "tcp", print))



get_flow_info:  tcpdump -n '(host 8.130.174.186 ) and tcp'
pid 
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

KeyboardInterrupt: 

In [ ]:
# monitor


In [32]:
i = "19:33:27.824632 IP 8.130.174.186.8024 > 192.168.3.156.58400: Flags [.], ack 183, win 12303, options [nop,nop,TS val 1461021786 ecr 2783603757], length 0"
part = i.split(": ")
print(part)
part1 = part[0].split(" ", 2)
part2 = part[1].split(", ")
print(part1, "\n", part2)
part3 = part1[2].split(" > ")
print(part3)
a, b = part3[0].rsplit(".", 1), part3[1].rsplit(".", 1)
src_ip, src_port = a[0], a[1]
dst_ip, dst_port = b[0], b[1]
print(src_ip, src_port)
print(dst_ip, dst_port)
timestmp = part1[0]
for j in part2:
    if "Flags" in j:
        print(j.split(" ")[1])
    elif "seq" in j:
        print(j.split(" ")[1].split(":"))
    elif "ack" in j:
        print(j.split(" ")[1])
    elif "options" in j:
        print(j.split(" ", 1)[1])
    elif "length" in j:
        print(j.split(" ")[1])


['19:33:27.824632 IP 8.130.174.186.8024 > 192.168.3.156.58400', 'Flags [.], ack 183, win 12303, options [nop,nop,TS val 1461021786 ecr 2783603757], length 0']
['19:33:27.824632', 'IP', '8.130.174.186.8024 > 192.168.3.156.58400'] 
 ['Flags [.]', 'ack 183', 'win 12303', 'options [nop,nop,TS val 1461021786 ecr 2783603757]', 'length 0']
['8.130.174.186.8024', '192.168.3.156.58400']
8.130.174.186 8024
192.168.3.156 58400
[.]
183
[nop,nop,TS val 1461021786 ecr 2783603757]
0
